# Unified Plotting

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#pi0scaling = {"val": 0, "suffix": "", "title": "No pi0 Scaling"} #no scaling
#pi0scaling = {"val": 1, "suffix": "_pi0flat", "title": "0.759 pi0 Scaling"} #flat scaling
pi0scaling = {"val": 2, "suffix": "_pi0e040", "title": r"$\pi^0$ scaling: [$1 - 0.40 \times E_\pi$]"} #energy-dependent


In [ ]:
from matplotlib import pyplot as plt

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
plt.rcParams.update(params)

from load_data_run123 import *
from unblinding_far_sideband import *

In [ ]:
import sys
import localSettings as ls
print(ls.main_path)
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
import scipy.stats

In [ ]:
#which_sideband = ['np_far'       ,"farsideband"] # far-sideband 1eNp datasets
#which_sideband = ['2plus_showers',"farsideband"] # far-sideband 2+ shower datasets
which_sideband = ['np_sb_comb',"farsideband"] # combined far-sideband 1eNp and 2+ shower datasets
#which_sideband = ['opendata'     ,'opendata']    # open datasets (5e19 / 1e19)

In [ ]:
samples = load_data_run123(which_sideband=which_sideband[0], return_plotter=False,\
                           pi0scaling=pi0scaling["val"],loadpi0variables=False)


In [ ]:
import plotter
import unblinding_far_sideband
import importlib
importlib.reload(plotter)
importlib.reload(unblinding_far_sideband)

In [ ]:
plotter_runbyrun = {}
for run in [1, 2, 3, 123, 12]:
    this_weights, this_pot = get_weights(run,dataset=which_sideband[1])
    plotter_runbyrun[run] = plotter.Plotter(samples, this_weights, pot=this_pot)

In [ ]:
TRAINVAR = ["shr_score","tksh_distance","tksh_angle",
            "shr_tkfit_dedx_max",
            "trkfit","trkpid",
            "subcluster","shrmoliereavg",
            "trkshrhitdist2","hits_ratio",
            "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y",
            "CosmicIPAll3D","CosmicDirAll3D"]

pvaluetypes = ['pvalue','pvaluediag','pvaluestatonly']

# Single Variable Plotter

In [ ]:
from unblinding_far_sideband import *

# choose sideband, preselection, and selection
# find defintions in unblinding_far_sideband.py
sdb_key = 'HiEext'
pre_key = 'NP'
sel_key = 'NPBDT'
#sdb_key = 'TwoPShrHiE'
#pre_key = 'NP'
#sel_key = 'TESTBDT02AllShr'#'NPLAllShr'
run = 123

sideband = sideband_categories[sdb_key]
preselection = preselection_categories[pre_key]
sel =  selection_categories[sel_key]

QUERY = ""
for query in [sideband['query'],preselection['query'],sel['query']]:
    if query is None: continue
    if QUERY: QUERY += ' and '
    QUERY += query
    if run in [1, 2, 3, 12]:
        QUERY += f' and run{run}==True'

#QUERY+=" and tksh_angle>0.98 and trkpid>-0.1"
#QUERY+=" and shr12_cos_p1_dstart>0.99"
#QUERY+=" and tksh_distance>6.0"
#QUERY+=" and n_showers_contained>1 and shr12_cos_p1_dstart>0.99"
#QUERY+=" and pi0_score>0.6"
#QUERY+=" and is_trk1bad==1"
#QUERY+=" and is_shr2splt==0 and is_trk1bad==0 and is_trk1embd==0 and is_trk2srtshr==0"
#QUERY+=" and (is_shr2splt==1 or is_trk1bad==1 or is_trk1embd==1 or is_trk2srtshr==1)"
print(QUERY)
        
#formatting can be improved..
TITLE = ""
for text in [f'Run {run}',' - ',sideband['title'],' - ',preselection['title'],'\n',sel['title'],' - ',pi0scaling["title"]]:
    if text is None: continue
    TITLE += text

#VARIABLE, BINS, RANGE, XTIT = 'topological_score',20,(0,1),r"topological score"
#VARIABLE, BINS, RANGE, XTIT = 'reco_e',21,(0.05, 2.15),r"reco energy"
VARIABLE, BINS, RANGE, XTIT = 'reco_e',7,(0.05, 2.85),r"reco energy"
#VARIABLE, BINS, RANGE, XTIT = 'reco_e',7,(0.05, 2.15),r"reco energy"
#VARIABLE, BINS, RANGE, XTIT = 'pi0_score',10,(0,1),r"pi0 score"
#VARIABLE, BINS, RANGE, XTIT = 'nonpi0_score',10,(0,1),r"non-pi0 score"
#VARIABLE, BINS, RANGE, XTIT = 'n_showers_contained',10,(-0.5, 9.5),"n showers contained"
#VARIABLE, BINS, RANGE, XTIT = 'shr_tkfit_dedx_max',10,(-0.5, 9.5),"variable name"
#VARIABLE, BINS, RANGE, XTIT = 'tksh_distance',20,(0, 40),"trk-shr distance"
#VARIABLE, BINS, RANGE, XTIT = 'trkshrhitdist2',20,(0, 20),"trk-shr hit dist Y"
#VARIABLE, BINS, RANGE, XTIT = 'trkpid',15,(-1, 1),"trk PID"
#VARIABLE, BINS, RANGE, XTIT = 'secondshower_Y_nhit',20,(0, 100),"secondshower Y nhit"
#VARIABLE, BINS, RANGE, XTIT = 'n_tracks_contained',10,(-0.5, 9.5),"n tracks contained"
#VARIABLE, BINS, RANGE, XTIT = 'tksh_angle',10,(-1, 1),"trk-shr angle"
#VARIABLE, BINS, RANGE, XTIT = 'ismerged',2,(-0.5, 1.5),"is merged"
#VARIABLE, BINS, RANGE, XTIT = 'merge_bestdot',20,(-1, 1),"merge bestdot"
#VARIABLE, BINS, RANGE, XTIT = 'merge_bestdist',20,(0, 40),"merge bestdist"
#VARIABLE, BINS, RANGE, XTIT = 'reco_e',17,(0.85,2.55),r"Reconstructed Energy [GeV]"

#VARIABLE, BINS, RANGE, XTIT = 'trkpid',21,(-1,1),"track LLR PID"
#VARIABLE, BINS, RANGE, XTIT = 'hits_ratio',20,(0,1),"shower hits/all hits"
#VARIABLE, BINS, RANGE, XTIT = 'shrmoliereavg',20,(0,50),"average Moliere angle [degrees]"
#VARIABLE, BINS, RANGE, XTIT = 'subcluster',20,(0,40),"N sub-clusters in shower"
#VARIABLE, BINS, RANGE, XTIT = 'trkfit',10,(0,1.0),"Fraction of Track-fitted points"
#VARIABLE, BINS, RANGE, XTIT = 'CosmicIPAll3D',20,(0,200),"CosmicIPAll3D [cm]"
#VARIABLE, BINS, RANGE, XTIT = 'tksh_distance',20,(0,40),"trk-shr distance [cm]"
#VARIABLE, BINS, RANGE, XTIT = 'shr_tkfit_dedx_max',15,(0,10),"shr tkfit dE/dx (max, 0-4 cm) [MeV/cm]"
#VARIABLE, BINS, RANGE, XTIT = 'tksh_angle',20,(-1,1),"cos(trk-shr angle)"
#VARIABLE, BINS, RANGE, XTIT = 'shr_trk_len',20,(0,400),"shr trk len [cm]"

#VARIABLE, BINS, RANGE, XTIT = 'trk_len',20,(0,200),"trk len [cm]"

#VARIABLE, BINS, RANGE, XTIT = 'shr12_cos_p1_dstart',20,(-1,1),"cos(shr1_dir,shr2_start-shr1_start)"
#VARIABLE, BINS, RANGE, XTIT = 'shr12_cos_p1_dstart',20,(0.8,1),"cos(shr1_dir,shr2_start-shr1_start)"

#VARIABLE, BINS, RANGE, XTIT = 'shr12_hitratio',20,(0,1),"Nhits shr2/shr1"
#VARIABLE, BINS, RANGE, XTIT = 'is_shr2splt',2,(-0.5,1.5),"is shr2splt"
#VARIABLE, BINS, RANGE, XTIT = 'is_trk1bad',2,(-0.5,1.5),"is trk1bad"
#VARIABLE, BINS, RANGE, XTIT = 'is_trk1embd',2,(-0.5,1.5),"is trk1embd"
#VARIABLE, BINS, RANGE, XTIT = 'is_trk2srtshr',2,(-0.5,1.5),"is trk2srtshr"
#VARIABLE, BINS, RANGE, XTIT = 'trk_distance',10,(0, 300),"trk-vtx distance"
#VARIABLE, BINS, RANGE, XTIT = 'trk_len',20,(0, 400),"trk len"

#RANGE = (0.6,1)

print(VARIABLE, BINS, RANGE, XTIT)

fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
    VARIABLE,   
    query=QUERY,
    kind="event_category",
    draw_sys=True,
    stacksort=3,
    title=XTIT,
    bins=BINS,
    range=RANGE,
)[0:3]

print('%s %.4f %.4f %.4f' % (VARIABLE, plotter_runbyrun[run].stats['pvaluestatonly'], plotter_runbyrun[run].stats['pvaluediag'], plotter_runbyrun[run].stats['pvalue']))

#ax1.set_ylim(0, ax1.get_ylim()[1]*1.8)

ax1.set_title(TITLE, loc='left')
#ax1.set_yscale('log')

plt.tight_layout()
fig.show()
#save_path = ls.plots_path+'PELEE_far_sideband_runbyrun/run_12/HiEadd_NPOneShr_Nm1NPL_pi0e040/'+VARIABLE
#fig.savefig(save_path + '_test_run3_trkpidcut.png', dpi=250)    
#fig.savefig(save_path + '.png', dpi=250)    
#plt.close()
 

In [ ]:
samples['data'].query(QUERY)[["run","sub","evt","trk2_id","trk2_id_test","n_tracks_contained","n_tracks_tot","shr2_id","shr2_id_test"]]

In [ ]:
#print(samples['data'].query(QUERY)[["run","sub","evt","reco_e","pi0_score"]])
samples['data'].query(QUERY)[["run","sub","evt","reco_e","n_showers_contained","n_tracks_contained","tksh_distance","shr_tkfit_dedx_max","trkpid","hits_ratio","trkfit","subcluster","shrmoliereavg","trkshrhitdist2","shr_trk_len","shr_score","pi0_score","nonpi0_score"]] 

In [ ]:
samples['data'].query(QUERY)[["run","sub","evt","reco_e","is_shr2splt","is_trk1bad","is_trk1embd","is_trk2srtshr","pi0_score","nonpi0_score"]]
#print(samples['data'].query(QUERY)[["run","sub","evt"]])#,"n_showers_contained","n_tracks_contained"
#,"trk2_protonenergy","trk2_energy","trk2_energy_cali"

In [ ]:
#samples['data'].query(QUERY)[["run","sub","evt","tksh_distance","reco_nu_vtx_x","reco_nu_vtx_y","reco_nu_vtx_z","shr_id","shr_start_x_v","shr_start_y_v","shr_start_z_v","trk_id","trk_score_v","trk_start_x_v","trk_start_y_v","trk_start_z_v","trk_end_x_v","trk_end_y_v","trk_end_z_v","trk_len_v","trk_llr_pid_score_v"]]#,"shr_phi_v","shr_theta_v","trk_phi_v","trk_theta_v"
samples['data'].query(QUERY+" and (run==13913 or run==14543 or run==16058)")[["run","sub","evt","n_tracks_contained","tksh_distance","trk_len","trkshrhitdist2","tksh_angle","hits_ratio","trkpid","shr_tkfit_dedx_max","CosmicIPAll3D","CosmicDirAll3D","shr_score","shrmoliereavg","subcluster","trkfit","secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y","pi0_score","nonpi0_score"]]

In [ ]:
#print(samples['data'].query(QUERY)[["run","sub","evt"]])
#QUERY=" and ".join([HIGH_ENERGY_EXT,BDTCQ])
print(QUERY)
print(len(samples['data'].query(QUERY)[["run","sub","evt","reco_e"]]))
pd.set_option('display.max_rows', 100)
print(samples['data'].query(QUERY)[["run","sub","evt","reco_e"]])

In [ ]:
#samples['ext'].query(unblinding_far_sideband.BDTCQ)[['run','evt','reco_e']]
samples['ext'].query(unblinding_far_sideband.BDTCQ)[['run','evt','reco_e','CosmicIPAll3D','trkpid','hits_ratio','trkfit','n_showers_contained','tksh_distance','tksh_angle']]

# Multiple variable plotting

In [ ]:
from unblinding_far_sideband import *

# choose sideband, preselection, and selection(s)
# find defintions in unblinding_far_sideband.py
#sdb_key = 'HiEext'
#pre_key = 'NPOneShr'
#sel_keys = ['TESTINTBDTCQ2'] #['None','NPVL','NPL','NPBDT']
sdb_key = 'TwoPShrHiE'
pre_key = 'NP'
sel_keys = ['TESTBDT05AllShr'] #['None','NPVLAllShr','NPLAllShr','NPBDTAllShr']
#sdb_key = 'None'
#pre_key = 'NP'
#sel_keys = ['None']#,'NPVL'
runs = [3,12] #[1,2,3,12,123]

sideband = sideband_categories[sdb_key]
preselection = preselection_categories[pre_key]

for run in runs:
    for sel_key in sel_keys:
        sel =  selection_categories[sel_key]
        print('\nRun %i, %s, %s, %s'%(run, sideband['dir'], preselection['dir'], sel['dir']))
        if (which_sideband[1] != "opendata"):
            this_folder = ls.plots_path+'PELEE_far_sideband_runbyrun/'
        else:
            this_folder = ls.plots_path+'PELEE_opendata/'
        this_folder += 'run_{}/'.format(run)
        this_folder += '{}_{}_{}'.format(sideband['dir'], preselection['dir'], sel['dir'])
        this_folder += (pi0scaling["suffix"]+'/')
        !mkdir -p $this_folder

        this_query = ""
        for query in [sideband['query'],preselection['query'],sel['query']]:
            if query is None: continue
            if this_query: this_query += ' and '
            this_query += query
        if run in [1, 2, 3, 12]:
            this_query += f' and run{run}==True'

        #formatting can be improved..
        this_title = ""
        for text in [f'Run {run}',' - ',sideband['title'],' - ',preselection['title'],'\n',sel['title'],' - ',pi0scaling["title"]]:
            if text is None: continue
            this_title += text

        pvalues = [[] for _ in range(len(pvaluetypes))]
        
        for plot_variable in plot_variables:
            VARIABLE, BINS, RANGE, XTIT = plot_variable[0:4]
            #if VARIABLE != 'tksh_angle': continue
            #if len(plot_variable)>4 or VARIABLE not in TRAINVAR: continue
            print(VARIABLE, BINS, RANGE, XTIT)
            fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
                VARIABLE,   
                query=this_query,
                kind="event_category",
                draw_sys=True,
                stacksort=3,
                title=XTIT,
                bins=BINS,
                range=RANGE,
            )[0:3]

            if len(plot_variable) == 6:
                if plot_variable[5] is True:
                    ax1.set_yscale('log')
            else:
                ax1.set_ylim(0, ax1.get_ylim()[1]*1.8) 

            ax1.set_title(this_title, loc='left')

            plt.tight_layout()
            save_path = this_folder+VARIABLE
            if len(plot_variable) >= 5:
                save_path += ('_' + plot_variable[4])
            fig.savefig(save_path + '.png', dpi=250)    
            plt.close()
            # store pvalues
            if len(plot_variable)==4 and VARIABLE in TRAINVAR:
                for idx, pvt in enumerate(pvaluetypes):
                    pvalues[idx].append(plotter_runbyrun[run].stats[pvt])
            #print ('%s %.04f'%(VARIABLE,plotter_runbyrun[run].stats['chisqstatonly']))
            print('%s %.4f %.4f %.4f' % (VARIABLE, plotter_runbyrun[run].stats['pvaluestatonly'], plotter_runbyrun[run].stats['pvaluediag'], plotter_runbyrun[run].stats['pvalue']))
        # make pvalue plots
        #print(pvalues)
        for idx, pvt in enumerate(pvaluetypes):
            if len(pvalues[idx])!=16: continue
            plt.figure(figsize=(9, 7))
            plt.hist(pvalues[idx], bins=10, range=(0,1))
            plt.title(this_title, loc='left')
            plt.xlabel(pvt)
            plt.ylabel("number of BDT input variables")
            plt.savefig(this_folder + pvt + '.png', dpi=250)
            plt.close()

In [ ]:
samples['data'].query(BDTCQ)[["run","sub","evt","reco_e","pi0_score","nonpi0_score","n_tracks_contained","reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z"]]